In [1]:
!pip install PySpark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PySpark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=52ba868ba65ea5b945d466da4a970fed838aa95e43035a96b1268de39592b7a8
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built PySpark


In [5]:
# Import necessary libraries
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

# create a spark session
spark = SparkSession.builder.appName("Decision Tree Classification").getOrCreate()

# Load the data into a DataFrame
train_data_load = spark.read.format("csv").option("header", "false").load("adult.data")
test_data_load= spark.read.format("csv").option("header", "false").load("adult.test")

# Add column names to the DataFrame
columns = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
           "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss",
           "hours_per_week", "native_country", "income"]
train_data_df = train_data_load.toDF(*columns)
test_data_df = test_data_load.toDF(*columns)

# Remove leading and trailing whitespaces from the categorical columns
categorical_columns = ["workclass", "education", "marital_status", "occupation",
                       "relationship", "race", "sex", "native_country", "income"]
for column in categorical_columns:
    train_data = train_data_df.withColumn(column, trim(col(column)))
    test_data = test_data_df.withColumn(column, trim(col(column)))


In [6]:
numerical_columns = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
for column in numerical_columns:
    train_data = train_data.withColumn(column, train_data[column].cast("integer"))
    test_data = test_data.withColumn(column, test_data[column].cast("integer"))

train_data_df.show()

+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
|age|        workclass| fnlwgt|    education|education_num|      marital_status|        occupation|  relationship|               race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
| 39|        State-gov|  77516|    Bachelors|           13|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|        2174|           0|            40| United-States| <=50K|
| 50| Self-emp-not-inc|  83311|    Bachelors|           13|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|           0|           0|        

In [7]:
test_data_df.show()

+---+-----------------+-------+-------------+-------------+-------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+-------+
|age|        workclass| fnlwgt|    education|education_num|     marital_status|        occupation|  relationship|               race|    sex|capital_gain|capital_loss|hours_per_week|native_country| income|
+---+-----------------+-------+-------------+-------------+-------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+-------+
| 25|          Private| 226802|         11th|            7|      Never-married| Machine-op-inspct|     Own-child|              Black|   Male|           0|           0|            40| United-States| <=50K.|
| 38|          Private|  89814|      HS-grad|            9| Married-civ-spouse|   Farming-fishing|       Husband|              White|   Male|           0|           0|         

In [8]:
# Add a label column to the data using StringIndexer
indexer = StringIndexer(inputCol="income", outputCol="label")
train_data = indexer.fit(train_data).transform(train_data)
test_data = indexer.fit(test_data).transform(test_data)

# Vectorize the features
categorical_features = ["workclass", "education", "marital_status", "occupation",
                        "relationship", "race", "sex", "native_country"]
numeric_features = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
assembler_inputs = [c + "_encoded" for c in categorical_features] + numeric_features
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

# Convert categorical columns to indices and one-hot encode them
indexers = [StringIndexer(inputCol=c, outputCol=c + "_index") for c in categorical_features]
encoders = [OneHotEncoder(inputCol=c + "_index", outputCol=c + "_encoded") for c in categorical_features]

# Build the decision tree classifier model
dt = DecisionTreeClassifier(featuresCol="features", labelCol="label")

# Chain indexers, encoders, assembler and the model in a Pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, dt])

# Fit the pipeline to the training data
model = pipeline.fit(train_data)

# Make predictions on the test data
predictions = model.transform(test_data)

# Evaluate the model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label")
accuracy = evaluator.evaluate(predictions)

print("Decision Tree Classifier Model accuracy: ", accuracy)


Decision Tree Classifier Model accuracy:  0.5651777176836972
